In [145]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from subprocess import check_output

# Pipeline at a glance:
load data -> cleaning/preprocessing -> feature eng -> format data for modelling -> fit model -> evaluate model -> generate submission file

## Step 1: Load data

In [146]:
df_train = pd.read_csv('../input/train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [147]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [148]:
df_test = pd.read_csv('../input/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [149]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## Step 2: Basic cleaning and preprocessing

In [150]:
# label encode sex
le = LabelEncoder()
le.fit(df_train.Sex)
df_train.loc[:,'Sex'] = le.transform(df_train.Sex)
df_test.loc[:,'Sex'] = le.transform(df_test.Sex)

In [151]:
# Fill in the missing values
mAge = pd.concat((df_train.Age, df_test.Age), axis=0).mean()
df_train.loc[:, 'Age'] = df_train.Age.fillna(mAge)
df_test.loc[:, 'Age'] = df_test.Age.fillna(mAge)

In [152]:
#Fill in missing values for embarked
df_train['Embarked'].fillna('S',inplace=True)
df_train.Embarked.isnull().any()

False

In [153]:
df_test['Embarked'].fillna('S',inplace=True)
df_test.Embarked.isnull().any()

False

## Step 3: Feature engineering

In [154]:
df_train['age_binned']=pd.cut(df_train['Age'],[0, 16, 32, 48, 64, 200], labels=[0,1,2,3,4], retbins=False)


In [155]:
df_test['age_binned']=pd.cut(df_train['Age'],[0, 16, 32, 48, 64, 200], labels=[0,1,2,3,4], retbins=False)


In [156]:
df_train['family_size'] = df_train.Parch + df_train.SibSp

In [157]:
df_test['family_size'] = df_test.Parch + df_test.SibSp

In [158]:
df_train['is_alone'] = np.where(df_train['family_size'] == 0, 1, 0)

In [159]:
df_test['is_alone'] = np.where(df_test['family_size'] == 0, 1, 0)

In [160]:
le.fit(df_train['Embarked'])

LabelEncoder()

In [161]:
list(le.classes_)

['C', 'Q', 'S']

In [162]:
df_train['Embarked'] = le.transform(df_train['Embarked'])

In [163]:
le.fit(df_test['Embarked'])
df_test['Embarked'] = le.transform(df_test['Embarked'])

In [164]:
df_train = df_train.drop(columns=['Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'])

In [165]:
df_test = df_test.drop(columns=['Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'])

## Step 4: Format data for modelling

In [166]:
df_train

,PassengerId,Survived,Pclass,Sex,Embarked,age_binned,family_size,is_alone
0,1,0,3,1,2,1,1,0
1,2,1,1,0,0,2,1,0
2,3,1,3,0,2,1,0,1
3,4,1,1,0,2,2,1,0
4,5,0,3,1,2,2,0,1
5,6,0,3,1,1,1,0,1
6,7,0,1,1,2,3,0,1
7,8,0,3,1,2,0,4,0
8,9,1,3,0,2,1,2,0
9,10,1,2,0,0,0,1,0


In [167]:
X = df_train.loc[:,['Embarked', 'Sex', 'Pclass', 'age_binned', 'family_size', 'is_alone']]
y = df_train.loc[:,'Survived']

In [168]:
dummy_df = pd.get_dummies(df_train[['Embarked', 'Sex', 'Pclass', 'age_binned', 'is_alone']])

In [169]:
# Split data X and y into train and val sets
X_test = df_test.loc[:,['Embarked', 'Sex', 'Pclass', 'age_binned', 'family_size', 'is_alone']]
X_train, X_valid, y_train, y_valid = train_test_split(
      X, y, test_size=0.2, random_state=42)

In [170]:
X_test.isnull().sum()

Embarked       0
Sex            0
Pclass         0
age_binned     0
family_size    0
is_alone       0
dtype: int64

## Step 5: Train (or fit) the model

In [171]:
# Fit logistic regression
LR = LogisticRegression()
LR.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Step 6: Evaluate the model

In [172]:
# define a scoring function
def acc(y: np.array, y_pred: np.array) -> float:
    return np.sum(y_pred==y)/len(y)

In [173]:
# Use model to predict validation set
y_pred_valid = LR.predict(X_valid)

In [174]:
print('Validation set accuracy', acc(y_valid, y_pred_valid), '%')

Validation set accuracy 0.8044692737430168 %


## Step 7: Generate Submission

In [175]:
# Predict for test set

y_pred_test = LR.predict(X_test)

# Create a Kaggle submission
sub = pd.DataFrame({'PassengerId': df_test['PassengerId'],
                    'Survived': y_pred_test})

sub.to_csv('basic_submission.csv', index=False)